<a href="https://colab.research.google.com/github/samehaisaa/Shape-Recognition-And-Descriptor-Analysis/blob/main/g%C3%A9od%C3%A9sique_3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install trimesh


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.8/704.8 kB 16.6 MB/s eta 0:00:00


In [ ]:
import trimesh
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import dijkstra
import plotly.graph_objs as go
from tqdm import tqdm


In [ ]:
seed_index1 = 13714  #bout du nez
filepath = '/content/F0001_AN01WH_F3Dsur.obj'


In [ ]:
def load_obj_mesh(filepath):
    mesh = trimesh.load(filepath)
    vertices = np.array(mesh.vertices)
    faces = np.array(mesh.faces)
    return vertices, faces
vertices, faces = load_obj_mesh(filepath)


ValueError: string is not a file: /content/F0001_AN01WH_F3Dsur.obj

In [ ]:
def compute_geodesic_distances(vertices, faces, seed_index, epsilon):
    rows, cols, data = [], [], []

    for face in tqdm(faces, desc="Building Graph"):
        for i in range(3):
            for j in range(i + 1, 3):
                vi, vj = face[i], face[j]
                dist = np.linalg.norm(vertices[vi] - vertices[vj])

                rows.extend([vi, vj])
                cols.extend([vj, vi])
                data.extend([dist, dist])

    num_vertices = len(vertices)
    graph = csr_matrix((data, (rows, cols)), shape=(num_vertices, num_vertices))

    distances, _ = dijkstra(csgraph=graph, directed=False, indices=seed_index, return_predecessors=True)

    return distances


In [ ]:
def plot_bipolar_contours(vertices, faces, sum_distances, target_distances, tolerance):
    x, y, z = vertices[:, 0], vertices[:, 1], vertices[:, 2]
    mesh3d = go.Mesh3d(
        x=x, y=y, z=z,
        i=faces[:, 0], j=faces[:, 1], k=faces[:, 2],
        intensity=sum_distances,
        colorscale='Viridis',
        colorbar_title='Sum of Geodesic Distances',
        showscale=False,
        opacity=0.8
    )
    seed_vertex1 = vertices[seed_index1]
    seed_marker1 = go.Scatter3d(
        x=[seed_vertex1[0]], y=[seed_vertex1[1]], z=[seed_vertex1[2]],
        mode='markers',
        marker=dict(size=10, color='blue', symbol='circle')
    )

    contours = []
    for target_distance in target_distances:
        selected_indices = np.where(np.isclose(distances, target_distance, atol=tolerance))[0]
        selected_vertices = vertices[selected_indices]

        contour = go.Scatter3d(
            x=selected_vertices[:, 0], y=selected_vertices[:, 1], z=selected_vertices[:, 2],
            mode='markers',
            marker=dict(size=4, color='red', symbol='circle')
        )
        contours.append(contour)

    # Layout
    layout = go.Layout(
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z')
        ),
        title=" Contours Based on equal Geodesic Distances",
        width=1000,
        height=800
    )

    fig = go.Figure(data=[mesh3d, seed_marker1] + contours, layout=layout)


    fig.show()




In [ ]:
epsilone = 0.1
vmin = 10
pas = 15
distances = compute_geodesic_distances(vertices, faces, seed_index1, epsilon=epsilone)
vmax = max(distances)
target_distances = [vmin + i * pas for i in range(int((vmax - vmin) // pas + 1))]


Building Graph: 100%|██████████| 61224/61224 [00:03<00:00, 18915.51it/s]


In [ ]:
plot_bipolar_contours(vertices, faces, distances, target_distances, tolerance=epsilone)
